___

<a href='https://github.com/eliasmelul/'> <img src='https://s3.us-east-2.amazonaws.com/wordontheamazon.com/BlueLogoNoBackground.png' style='width: 110px;' align='right' /></a>
# Capstone Project: Looking for my Schitt's Creek
___
<h4 align="right">by Elias Melul, Data Scientist </h4> 



### Business Understanding

The purpose of this capstone project is quite simple, and one that many people have encountered.
_______________________________________________________________________________________________________________________

Most of us have heard of New York City, Boston, San Francisco, Chicago, Miami... all very different cities with a lot to offer. However, there are many incredible cities in the United States, cities that we might not know much about, nor we learn about until someone introduces us to it!

As an international student in the US, I constantly wonder if my desire to go to one of the big cities (NYC, Boston, San Fran. etc.) is valid and if there are other places in the US that I may not know of. I go to Duke University, and I really liked Raleigh-Durham. Had I not gone there, I would not have known!

-------------------

So I am going to classify the cities based on an amalgalm of features to see which one are statistically most similar than others. I will also attempt to build a recommendator system that takes your favorite cities (and least favorites!) and the rating you give them and return other cities with similar characteristics. Feel free to try it out!

### Data Acquisition

In order to create a good clustering model and a competent recommender system, I need information from cities in the USA, and lots of it! 

----
**Which cities will I include?**

I found [this](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population) Wikipedia site that includes a table of the top 314 most populated cities in the USA. We will use these cities to scrape the web, leverage FourSquare API, and gather as much data as possible to feed to our classificator model and our recommender system.

----
**What data will be collected and how?**

There is a CRAZY amount of data available on the web about all cities, but these can be very dispar and from numerous websites. Lucky me, I found [this](https://datausa.io/) website that contains most the information I need (thank you Deloitte and Datawheel!), so I will selectively choose to scrape metrics that I believe might be important. I will then combine this information with FourSquare information about venues and others to complete the dataframe and begin modeling!

**Some of the variables scraped are:**

* Population and Population Change (Year to Year)
* Poverty Rate
* Median Age
* Median Household Income and Median Household Income Change (Year to Year)
* Number of Employees and Number of Employees Change (Year to Year)
* Median Property Value and Median Property Value Change (Year to Year)
* Average Male and Female Salary, and a ratio of Average Male to Female Salary
* Gini coefficient in 2017 and 2018, as well as it's change (Year to Year)
* Ratio of Patients to Clinicians (county-wise)
* Foreign-born population percentage
* Citizen population percentage
* Total degrees awarded in 2018 (higher education)
* Male to Female ratio of awarded degrees
* Number of degrees per capita
* Number of households in city
* Population per household (people per household)
* Homeownership Percentage (Rent vs Own)
* Average Commute Time (minutes)

##### Import Libraries

In [1]:
import pandas as pd # import pandas for dataframes
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import re

##### Load Wikipedia Data for City Names

In [2]:
all_cities = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')[4]
all_cities.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York[d],New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


There are three clear problems with the imported table above:
1. Some of the city names have square brakets due to footnotes included in the table. We must get rid of them.
2. The state column is not abbreviated. Why is this an issue? Because of the url format datausa.io has. This will become clear in the next section.
3. The format of the values for each city is not adequate. We need to convert number to numeric.

**Lets fix problem 1: deleting everything within square brakets in the City column.**

In [3]:
for i, row in all_cities.iterrows():
    ifor_val = re.sub(r'\[.*\]', '', row.City)
    all_cities.at[i,'City'] = ifor_val
all_cities.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York,New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


Lets also rename the columns and subset the data to the useful columns.

In [4]:
all_cities.rename(columns={'2018rank':'Rank',
                  'State[c]':'State',
                  '2018estimate':'Population_2018',
                  '2010Census':'Population_2010',
                  '2016 land area.1':'Land_Area_km2',
                  '2016 population density.1':'Population_Density_per_km2'}, inplace=True)

all_cities = all_cities[['City','State','Land_Area_km2','Population_Density_per_km2']]
all_cities.head()

,City,State,Land_Area_km2,Population_Density_per_km2
0,New York,New York,780.9 km2,"10,933/km2"
1,Los Angeles,California,"1,213.9 km2","3,276/km2"
2,Chicago,Illinois,588.7 km2,"4,600/km2"
3,Houston,Texas,"1,651.1 km2","1,395/km2"
4,Phoenix,Arizona,"1,340.6 km2","1,200/km2"


**Lets fix problem 2: Adding State abbreviations**

To add the abbreviations, we must get a list of abbreviations and their respective states spelled out. Luckily, Wikipedia has a site listing all the US abbreviations! We will first import them, and then add the appropiate abbreviation to each City in our dataframe.

In [5]:
abb_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations', skiprows=11)[0]
abb_df = abb_df[['United States of America', 'Unnamed: 5']]
abb_df = abb_df.rename(columns={'United States of America':'State','Unnamed: 5':'State_Abb'})
abb_df.head()

,State,State_Abb
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [6]:
all_cities = all_cities.merge(abb_df, on='State')
all_cities.tail()

,City,State,Land_Area_km2,Population_Density_per_km2,State_Abb
311,Columbia,South Carolina,345.8 km2,388/km2,SC
312,North Charleston,South Carolina,190.9 km2,573/km2,SC
313,Fargo,North Dakota,127.7 km2,950/km2,ND
314,Manchester,New Hampshire,85.7 km2,"1,289/km2",NH
315,Billings,Montana,113.2 km2,975/km2,MT


**Lets fix problem 3: Formatting column contents**

In [7]:
for i, row in all_cities.iterrows():
    all_cities.at[i,'Land_Area_km2'] = float(row.Land_Area_km2[0:-4].replace(',',''))
    all_cities.at[i,'Population_Density_per_km2'] = float(row.Population_Density_per_km2[0:-4].replace(',',''))
all_cities.head()

,City,State,Land_Area_km2,Population_Density_per_km2,State_Abb
0,New York,New York,780.9,10933,NY
1,Buffalo,New York,104.6,2455,NY
2,Rochester,New York,92.7,2253,NY
3,Yonkers,New York,46.6,4307,NY
4,Syracuse,New York,64.7,2214,NY


In [231]:
#Lets also create a column with standard city namage: New York, NY
all_cities = all_cities.rename(columns={'City':'City Name'})
all_cities['City'] = [row['City Name']+", "+row['State_Abb'] for i,row in all_cities.iterrows()]
all_cities.head()

,City Name,State,Land_Area_km2,Population_Density_per_km2,State_Abb,City
0,New York,New York,780.9,10933,NY,"New York, NY"
1,Buffalo,New York,104.6,2455,NY,"Buffalo, NY"
2,Rochester,New York,92.7,2253,NY,"Rochester, NY"
3,Yonkers,New York,46.6,4307,NY,"Yonkers, NY"
4,Syracuse,New York,64.7,2214,NY,"Syracuse, NY"


Great! Now that these problems have been fixed, we can proceed to scraping the web for more data! 

---
For that, we will need a list of all the cities and their respective state abbreviation. Why? Because the path to our scrapping website is as follows:

    /city-name-state_abbretiation
    
    For example:
    /new-york-ny
 

However, not all URLs are that clear. Some contain paths that include 'metropolitan area' or some require a 'county'. For that reason, we will generate URLs that also include the following forms:

    /new-york-ny-metro-area
    /new-york-county-ny

Once we generate these URLs, we will check if they are valid, extract another list of only valid URLs and use those to scrape data.

##### Generating the URLs

In [130]:
cities_list = all_cities['City Name'] + " "+all_cities.State_Abb
cities_list = list(cities_list)
cities_list= [i.lower().replace(' ','-') for i in cities_list]

In [131]:
# Add -metro-area
cities_metro = [i+'-metro-area' for i in cities_list]

In [132]:
# #Add county possible urls
# county_list = all_cities['City Name'] +"-county-"+all_cities.State_Abb
# county_list = [i.lower().replace(' ','-') for i in county_list]

In [133]:
# Combine the lists
cities_list=cities_list+cities_metro#+county_list

In [134]:
#Concatenate the domain to the generated path to recreate all the URLS
urls_cities = ["https://datausa.io/profile/geo/"+i for i in cities_list]

In [135]:
len(urls_cities)

632

We have generated 948 possible URLs to scrap the data.

Let's check how many of them are valid and keep those for scraping.

In [136]:
#Check if url connections exist
checker = []
for url in urls_cities:
    c = requests.get(url)
    soup = BeautifulSoup(c.content, features='lxml')
    res = soup.find_all("h4",{"class":"pt-non-ideal-state-title"})
    if (len(res) > 0):
        exist = 0
    else:
        try:
            tes = soup.find("div",{"class":"content-container"})
            tes = tes.get_text()
            if (tes == 'N/A'):
                exist = 0
            else:
                exist =1
        except: 
            exist=1
            pass
           
    tempdic = {"URL":url,
              "Exist":exist}
    checker.append(tempdic)

checker = pd.DataFrame(checker)
checker.head()

,URL,Exist
0,https://datausa.io/profile/geo/new-york-ny,1
1,https://datausa.io/profile/geo/buffalo-ny,1
2,https://datausa.io/profile/geo/rochester-ny,1
3,https://datausa.io/profile/geo/yonkers-ny,1
4,https://datausa.io/profile/geo/syracuse-ny,1


In [137]:
existing = checker[checker.Exist==1]
existing.shape

(373, 2)

In [138]:
nonexisting = checker[checker.Exist==0]
nonexisting.shape

(259, 2)

In [139]:
#Create a list with the existing
urls_cities = existing['URL'].to_list()

As we can see, there are 373 URLs from our generated URL list. 

If we recall, there were only 314 cities on our list... it seems likely that have a lot repeat cities/areas due to the 'County' and 'Metro-Area' paths added. We will deal with this issue after scraping.

---
For now, let's have some fun!

##### Scraping dataUSA

For this section, we have defined everyhing as functions, so that we can easily iterate over all the URLs. 

---
The first function simply retrieves the text data from the HTML section specified by the class types and names.

---

The second function uses the first function to retrieve each metric wanted for each city as strings, and transforms such to the type desired. In this case, all the information is numeric except the name of the city. A dictionary will be returned.


In [147]:
def get_text_from_class(soup,classname,type1="div",type2="class"):
    city_content = soup.find(type1, {type2:classname})
    city_info = city_content.find_all('p')
    content_raw = []
    for i in city_info:
        i = str(i)
        removefirst = i[3:-4]
        content_raw.append(removefirst)
    return(content_raw)

In [148]:
def get_basic_city_info(url):
    html = requests.get(url=url)
    soup = BeautifulSoup(html.content, features='lxml')
    
    ###### Get content from dashboard
    #Name of the city
    city_name = soup.find("p").get_text()
    
    #Raw dashboard content
    content_raw = get_text_from_class(soup, "profile-stats")
    
    #Population of City
    Population = content_raw[1]
    
    if Population[-1:] == "M":
        Population = float(Population[0:-1])*1000000
    else:
        Population = float(Population.replace(',',''))
    
    #Population Change YTY
    Population_Change = content_raw[2]
    if Population_Change[-7:] == 'decline':
        Population_Change = float(Population_Change[0:-9])*-1
    else:
        Population_Change = float(Population_Change[0:-8])
        
    #Poverty Rate
    Poverty_Rate = float(content_raw[4][0:-1])
    
    #Median Age
    Median_Age = float(content_raw[7])
    
    #Median Household Income and Change
    Median_Household_Income = float(content_raw[10][1:].replace(',',''))
    Median_Household_Income_Change = content_raw[11]
    if Median_Household_Income_Change[-7:]=='decline':
        Median_Household_Income_Change = float(Median_Household_Income_Change[0:-9])*-1
    else:
        Median_Household_Income_Change = float(Median_Household_Income_Change[0:-8])
    
    #Number of Employees
    Number_Employees = content_raw[13]
    if Number_Employees[-1:] == "M":
        Number_Employees = float(Number_Employees[0:-1])*1000000
    else:
        Number_Employees = float(Number_Employees.replace(',',''))
    
    #Change Number of Employees
    Number_Employees_Change = content_raw[14]
    if Number_Employees_Change[-7:]=='decline':
        Number_Employees_Change = float(Number_Employees_Change[0:-9])*-1
    else:
        Number_Employees_Change = float(Number_Employees_Change[0:-8])
    
    #Median Property Value
    Median_Property_Value = content_raw[16][1:]
    if Median_Property_Value[-1:] == "M":
        Median_Property_Value = float(Median_Property_Value[0:-1])*1000000
    else:
        Median_Property_Value = float(Median_Property_Value.replace(',',''))
    
    #Median Property Value Change
    Median_Property_Value_Change = content_raw[17]
    if Median_Property_Value_Change[-7:]=='decline':
        Median_Property_Value_Change = float(Median_Property_Value_Change[0:-9])*-1
    else:
        Median_Property_Value_Change = float(Median_Property_Value_Change[0:-8])
    
    #Wage Distribution across genders
    wage_gender = get_text_from_class(soup, "topic income_gender TextViz")
    avg_male_salary = float(wage_gender[2][1:].replace(',',''))
    avg_female_salary = float(wage_gender[5][1:].replace(',',''))
    gender_salary_ratio = avg_male_salary/avg_female_salary
  
    #Gini coefficient distribution
    gini_coeff = get_text_from_class(soup, "topic income_distro TextViz")
    gini_2018 = float(gini_coeff[2])
    gini_2017 = float(gini_coeff[4])
    gini_change_percent = (gini_2018-gini_2017)*100/gini_2017
    
    #Health Ratio (Patients to Clinicians)
    health_ratio = get_text_from_class(soup, "topic clinician_patient_ratio TextViz")
    health_ratio = float(health_ratio[2].replace(' to 1','').replace(',',''))
    
    #Foreign Born Population
    foreign_ratio = get_text_from_class(soup, "topic foreign_born TextViz")
    foreign_ratio = float(foreign_ratio[1][:-1].replace(',',''))
    
    #Citizen Ratio
    citizen_ratio = get_text_from_class(soup, "topic citizenship TextViz")
    citizen_ratio = float(citizen_ratio[1][:-1].replace(',',''))
    
    #Degrees awareded
    # There seem to be some cities without education infomration... we need to handle this exception
    try:
        degrees_awarded = get_text_from_class(soup, "topic edu_gender TextViz")
        degrees_men = float(degrees_awarded[1].replace(',',''))
        degrees_women = float(degrees_awarded[3].replace(',',''))
        degrees_gender_ratio_M2F = degrees_men/degrees_women 
        total_degrees = degrees_men+degrees_women
        degrees_per_capita = total_degrees/Population
    except:
        total_degrees = 0
        degrees_gender_ratio_M2F = 1
        degrees_per_capita = None
    
    #Number of Households
    households = get_text_from_class(soup, "topic household_income TextViz")
    households = households[5]
    if households[-1:] == "M":
        households = float(households[0:-1])*1000000
    elif households[-1:]=="k":
        households = float(households[0:-1])*1000
    else:
        households = float(households.replace(',',''))
    people_per_house = Population/households
    
    #Rent vs Ownership of Homes
    rent_own = get_text_from_class(soup, "topic rent_own TextViz")
    rent_own = float(rent_own[1][:-1])
    
    #Communite time
    commute_time = get_text_from_class(soup, "topic commute_time TextViz")
    commute_time = float(commute_time[1][:-8])
    
    basic_information = {'City':city_name,
                        'Population':Population,
                        'Population Change':Population_Change,
                        'Poverty Rate':Poverty_Rate,
                        'Median Age':Median_Age,
                        'Median Household Income':Median_Household_Income,
                        'Median Household Income Change':Median_Household_Income_Change,
                        'Number Employees':Number_Employees,
                        'Number Employees Change':Number_Employees_Change,
                        'Median Property Value':Median_Property_Value,
                        'Median Property Value Change':Median_Property_Value_Change,
                        'Average Male Salary':avg_male_salary,
                        'Average Female Salary':avg_female_salary,
                        'Gender Salary Ratio M2F':gender_salary_ratio,
                        'Gini 2018':gini_2018,
                        'Gini 2017':gini_2017,
                        'Gini Change':gini_change_percent,
                        'Patient to Clinician Ratio':health_ratio,
                        'Foreign Born Population Ratio':foreign_ratio,
                        'Citizens Percentage':citizen_ratio,
                        'Total Degrees':total_degrees,
                        'Degrees Ratio M2F':degrees_gender_ratio_M2F,
                        'Degrees per Capita':degrees_per_capita,
                        'Households':households,
                        'People Per House':people_per_house,
                        'Homeownership':rent_own,
                        'Commute Time':commute_time}
    
    
    return basic_information

##### Functions Use Example

We will use NYC to see how the functions work and how the output looks.

In [149]:
url = 'https://datausa.io/profile/geo/new-york-ny'

In [150]:
get_basic_city_info(url)

{'City': 'New York, NY',
 'Population': 8400000.0,
 'Population Change': -2.6,
 'Poverty Rate': 19.6,
 'Median Age': 36.9,
 'Median Household Income': 63799.0,
 'Median Household Income Change': 4.8,
 'Number Employees': 4090000.0,
 'Number Employees Change': -2.21,
 'Median Property Value': 645100.0,
 'Median Property Value Change': 5.84,
 'Average Male Salary': 81735.0,
 'Average Female Salary': 63914.0,
 'Gender Salary Ratio M2F': 1.2788277998560567,
 'Gini 2018': 0.493,
 'Gini 2017': 0.497,
 'Gini Change': -0.8048289738430591,
 'Patient to Clinician Ratio': 1553.0,
 'Foreign Born Population Ratio': 36.9,
 'Citizens Percentage': 84.3,
 'Total Degrees': 147053.0,
 'Degrees Ratio M2F': 0.608367056764738,
 'Degrees per Capita': 0.017506309523809525,
 'Households': 3180000.0,
 'People Per House': 2.641509433962264,
 'Homeownership': 32.8,
 'Commute Time': 40.1}

**Now that we have defined these functions, lets iterate over all our valid URLs to extract the specified metrics**

While running through the iterations of each city URL, there were some probleatic valid URLs. Take [Louisville, KY](https://datausa.io/profile/geo/louisville-ky) as an example. It does not contain the dashboard we typically use to extract some of the basic information. In addition, if has a bunch of NAs in the values we scrape from other parts of the website. Hence we are going to take these problematic URLs off the list.

---
**Dropping specific URLs**

In [151]:
urls_cities.remove('https://datausa.io/profile/geo/louisville-ky')

ValueError: list.remove(x): x not in list

**Iterate over all valid URLs and extract all the information specified in our function**

In [152]:
cities_basic = []
for url in urls_cities:
    data = get_basic_city_info(url)
    cities_basic.append(data)

In [153]:
len(cities_basic)

372

In [217]:
cities = pd.DataFrame(cities_basic)
print(cities.shape)
cities.head()

(372, 27)


,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,...,Patient to Clinician Ratio,Foreign Born Population Ratio,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time
0,"New York, NY",8400000.0,-2.6000,19.6,36.9,63799.0,4.80,4090000.0,-2.210,645100.0,...,1553.0,36.90,84.3,147053.0,0.608367,0.017506,3180000.0,2.641509,32.8,40.1
1,"Buffalo, NY",256322.0,-0.8780,30.9,33.3,37359.0,7.31,114478.0,-4.780,98300.0,...,1243.0,9.49,93.9,8053.0,0.684376,0.031418,109000.0,2.351578,41.6,18.9
2,"Rochester, NY",209463.0,-0.3940,33.1,31.7,32347.0,2.09,90549.0,1.620,79400.0,...,966.0,8.71,94.9,3985.0,0.933527,0.019025,86200.0,2.429965,36.5,19.5
3,"Yonkers, NY",200999.0,0.6380,16.4,38.8,62399.0,1.84,94272.0,2.710,380100.0,...,728.0,30.60,87.0,561.0,0.342105,0.002791,74400.0,2.701599,47.3,33.3
4,"Syracuse, NY",144405.0,0.0381,32.6,30.6,34716.0,6.15,58724.0,0.118,91100.0,...,1050.0,12.50,92.1,8707.0,0.784953,0.060296,55600.0,2.597212,38.3,17.8


Amazing! We have scraped information about 372 cities and/or metropolitan areas - all containing 26 metrics that may be useful in our segmentation and recommendation problem.

---
Since there is some information we may want to use from the table we got from Wikipedia to extract the city names, we need to change the city name format to separate state abbreviation and city name. Lets go ahead and do that.

In [218]:
for i, row in cities.iterrows():
    cities.at[i,'City Name'] = row.City[:-4]
    cities.at[i,'State_Abb'] = row.City[-2:]

Before we join the two tables, there is a critical issue to resolve: we have more rows in the scraped dataframe than we have cities on our list!

In [219]:
dupli = cities[cities.duplicated()] #Gets duplicated rows - where the WHOLE row is duplicated
dupli

,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,...,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time,City Name,State_Abb


In [220]:
dupli = cities[cities['City'].duplicated(keep=False)]
dupli

,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,...,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time,City Name,State_Abb
2,"Rochester, NY",209463.0,-0.3940,33.10,31.7,32347.0,2.09,90549.0,1.620,79400.0,...,94.9,3985.0,0.933527,0.019025,86200.0,2.429965,36.5,19.5,Rochester,NY
4,"Syracuse, NY",144405.0,0.0381,32.60,30.6,34716.0,6.15,58724.0,0.118,91100.0,...,92.1,8707.0,0.784953,0.060296,55600.0,2.597212,38.3,17.8,Syracuse,NY
9,"Fresno, CA",530073.0,0.5030,28.40,31.5,49813.0,2.50,219244.0,0.037,257200.0,...,90.3,9907.0,0.558440,0.018690,171000.0,3.099842,44.2,21.8,Fresno,CA
22,"Modesto, CA",210166.0,0.7930,17.00,35.3,54024.0,5.94,86370.0,3.270,236200.0,...,90.8,2805.0,0.482558,0.013347,71400.0,2.943501,54.6,26.0,Modesto,CA
39,"Salinas, CA",156811.0,0.5910,17.20,30.2,54864.0,4.83,67547.0,0.551,342100.0,...,70.8,1919.0,0.521808,0.012238,40600.0,3.862340,44.5,22.6,Salinas,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,"Salt Lake City, UT",1220000.0,1.6200,10.30,32.8,73730.0,3.10,632724.0,0.953,329200.0,...,92.2,40513.0,0.630367,0.033207,402000.0,3.034826,67.4,21.4,Salt Lake City,UT
368,"Sioux Falls, SD",250564.0,1.3100,9.21,35.0,62047.0,4.07,139089.0,2.090,171000.0,...,96.8,2238.0,0.689057,0.008932,98200.0,2.551568,67.3,17.9,Sioux Falls,SD
369,"Jackson, MS",583080.0,0.7410,18.50,36.7,55700.0,6.23,269835.0,2.820,151400.0,...,98.6,9590.0,0.661757,0.016447,210000.0,2.776571,67.8,23.8,Jackson,MS
370,"Columbia, SC",833165.0,0.9890,15.60,37.0,53765.0,-1.31,389276.0,-0.359,156800.0,...,97.5,14477.0,0.678687,0.017376,315000.0,2.644968,67.1,24.1,Columbia,SC


In [221]:
dupli = dupli.sort_values(by='Population', ascending=False)
dupli = dupli.reset_index().drop('index', axis=1)
dupli.head()

,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,...,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time,City Name,State_Abb
0,"Pittsburgh, PA",2320000.0,-0.370,11.8,43.1,59710.0,2.030,1170000.0,0.509,160300.0,...,98.2,39982.0,0.763030,0.017234,1010000.0,2.297030,69.6,25.4,Pittsburgh,PA
1,"Columbus, OH",2110000.0,1.340,14.0,36.1,64052.0,0.452,1070000.0,1.040,196700.0,...,95.5,33833.0,0.778905,0.016035,802000.0,2.630923,61.9,22.5,Columbus,OH
2,"Jacksonville, FL",1530000.0,1.970,13.9,38.8,60238.0,2.600,727956.0,2.880,217200.0,...,95.8,18224.0,0.709087,0.011911,575000.0,2.660870,63.5,25.2,Jacksonville,FL
3,"Oklahoma City, OK",1400000.0,0.918,14.6,35.3,57485.0,2.180,683637.0,2.550,160900.0,...,94.9,22794.0,0.757305,0.016281,522000.0,2.681992,63.1,21.9,Oklahoma City,OK
4,"Richmond, VA",1300000.0,0.160,12.4,38.8,67703.0,0.103,658555.0,1.150,246100.0,...,95.6,16092.0,0.637363,0.012378,497000.0,2.615694,65.6,23.7,Richmond,VA


By sorting the data by population descending and reseting the index, we can now remove all duplicate rows based on their position. In other words, we can keep the first duplicate record - the metropolitan area one most likely.

In [222]:
dupli = dupli.drop_duplicates(subset='City',keep='first')
print(dupli.shape)
dupli.head()

(68, 29)


,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,...,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time,City Name,State_Abb
0,"Pittsburgh, PA",2320000.0,-0.370,11.8,43.1,59710.0,2.030,1170000.0,0.509,160300.0,...,98.2,39982.0,0.763030,0.017234,1010000.0,2.297030,69.6,25.4,Pittsburgh,PA
1,"Columbus, OH",2110000.0,1.340,14.0,36.1,64052.0,0.452,1070000.0,1.040,196700.0,...,95.5,33833.0,0.778905,0.016035,802000.0,2.630923,61.9,22.5,Columbus,OH
2,"Jacksonville, FL",1530000.0,1.970,13.9,38.8,60238.0,2.600,727956.0,2.880,217200.0,...,95.8,18224.0,0.709087,0.011911,575000.0,2.660870,63.5,25.2,Jacksonville,FL
3,"Oklahoma City, OK",1400000.0,0.918,14.6,35.3,57485.0,2.180,683637.0,2.550,160900.0,...,94.9,22794.0,0.757305,0.016281,522000.0,2.681992,63.1,21.9,Oklahoma City,OK
4,"Richmond, VA",1300000.0,0.160,12.4,38.8,67703.0,0.103,658555.0,1.150,246100.0,...,95.6,16092.0,0.637363,0.012378,497000.0,2.615694,65.6,23.7,Richmond,VA


In [223]:
cities = cities.drop_duplicates(subset='City', keep=False)
print(cities.shape)
cities.head()

(236, 29)


,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,...,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time,City Name,State_Abb
0,"New York, NY",8400000.0,-2.600,19.6,36.9,63799.0,4.80,4090000.0,-2.210,645100.0,...,84.3,147053.0,0.608367,0.017506,3180000.0,2.641509,32.8,40.1,New York,NY
1,"Buffalo, NY",256322.0,-0.878,30.9,33.3,37359.0,7.31,114478.0,-4.780,98300.0,...,93.9,8053.0,0.684376,0.031418,109000.0,2.351578,41.6,18.9,Buffalo,NY
3,"Yonkers, NY",200999.0,0.638,16.4,38.8,62399.0,1.84,94272.0,2.710,380100.0,...,87.0,561.0,0.342105,0.002791,74400.0,2.701599,47.3,33.3,Yonkers,NY
5,"Los Angeles, CA",3990000.0,-0.232,20.4,35.8,62474.0,3.78,2060000.0,0.509,682400.0,...,80.7,78532.0,0.691261,0.019682,1380000.0,2.891304,36.3,30.3,Los Angeles,CA
6,"San Diego, CA",1430000.0,0.459,14.5,35.4,79646.0,3.89,715998.0,-0.985,654700.0,...,87.8,51571.0,0.694575,0.036064,514000.0,2.782101,46.8,22.5,San Diego,CA


In [224]:
cities = pd.concat([cities,dupli])
cities.shape

(304, 29)

Now that we have our data in the right format, lets do an inner join on the two dataframes that we want to combine on City and State abbreviation.

**Joining the Two Data Frames**

In [225]:
cities = cities.merge(all_cities,how='inner', left_on=['City Name'], right_on=['City'])
print(cities.shape)
cities.head()

,City_x,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,...,People Per House,Homeownership,Commute Time,City Name,State_Abb_x,City_y,State,Land_Area_km2,Population_Density_per_km2,State_Abb_y
0,"New York, NY",8400000.0,-2.600,19.6,36.9,63799.0,4.80,4090000.0,-2.210,645100.0,...,2.641509,32.8,40.1,New York,NY,New York,New York,780.9,10933,NY
1,"Buffalo, NY",256322.0,-0.878,30.9,33.3,37359.0,7.31,114478.0,-4.780,98300.0,...,2.351578,41.6,18.9,Buffalo,NY,Buffalo,New York,104.6,2455,NY
2,"Yonkers, NY",200999.0,0.638,16.4,38.8,62399.0,1.84,94272.0,2.710,380100.0,...,2.701599,47.3,33.3,Yonkers,NY,Yonkers,New York,46.6,4307,NY
3,"Los Angeles, CA",3990000.0,-0.232,20.4,35.8,62474.0,3.78,2060000.0,0.509,682400.0,...,2.891304,36.3,30.3,Los Angeles,CA,Los Angeles,California,1213.9,3276,CA
4,"San Diego, CA",1430000.0,0.459,14.5,35.4,79646.0,3.89,715998.0,-0.985,654700.0,...,2.782101,46.8,22.5,San Diego,CA,San Diego,California,842.3,1670,CA


for i, row in c

In [230]:
for i, row in dupli.iterrows():
    print(row.City)

Pittsburgh, PA
Columbus, OH
Jacksonville, FL
Oklahoma City, OK
Richmond, VA
Salt Lake City, UT
Rochester, NY
Tucson, AZ
Fresno, CA
Tulsa, OK
Albuquerque, NM
Knoxville, TN
El Paso, TX
Columbia, SC
Baton Rouge, LA
Dayton, OH
Colorado Springs, CO
Akron, OH
Madison, WI
Syracuse, NY
Wichita, KS
Springfield, MA
Toledo, OH
Jackson, MS
Modesto, CA
Lafayette, LA
Port St. Lucie, FL
Springfield, MO
Huntsville, AL
Corpus Christi, TX
Fort Wayne, IN
Salinas, CA
Salem, OR
Mobile, AL
Anchorage, AK
Savannah, GA
Tallahassee, FL
Fayetteville, NC
Montgomery, AL
Ann Arbor, MI
Peoria, IL
Rockford, IL
Lincoln, NE
Boulder, CO
Green Bay, WI
Lubbock, TX
Greeley, CO
Wilmington, NC
Gainesville, FL
Laredo, TX
Cedar Rapids, IA
Waco, TX
Amarillo, TX
Sioux Falls, SD
Tuscaloosa, AL
Topeka, KS
Tyler, TX
Rochester, MN
Las Cruces, NM
Springfield, IL
Columbia, MO
Abilene, TX
Billings, MT
Midland, TX
Pueblo, CO
Odessa, TX
Wichita Falls, TX
San Angelo, TX
